# Qiskit Prototype: Threshold Grover and Dürr–Høyer Minimum Finding

Objectives:
- Find pumps over a threshold per variable (Grover threshold oracle).
- Find the healthiest pump (argmin) per variable using a simplified Dürr–Høyer loop.

Notes:
- Variables are normalized to [0, 1]; lower values indicate healthier equipment.
- Early prototype embeds values classically into the oracle by marking indices directly (no QRAM yet).
- Keep N small (e.g., 8–32) for demonstration; circuits scale with the number of marked indices.

In [ ]:
# If needed, install/upgrade Qiskit packages in your virtual environment:
# %pip install -U qiskit qiskit-aer

from math import ceil, log2, pi, sqrt
from collections import Counter
import random

from qiskit import QuantumCircuit
from qiskit import transpile
try:
    from qiskit_aer import Aer
except Exception:
    # Fallback for environments where qiskit-aer is provided via qiskit
    from qiskit import Aer

print("Qiskit backend loaded:", Aer)


In [ ]:
def num_qubits_for_n(n_items: int) -> int:
    return max(1, ceil(log2(max(1, n_items))))

def index_to_bits(i: int, n_qubits: int):
    return [int(b) for b in format(i, f"0{n_qubits}b")]


In [ ]:
def phase_oracle_gate(n_index: int, marked_indices):
    """
    Returns a Gate that flips the phase of |i> states for i in marked_indices using an ancilla in |->.
    Circuit expects n_index index qubits and 1 ancilla qubit as the last qubit.
    """
    qc = QuantumCircuit(n_index + 1, name="Oracle")
    for idx in marked_indices:
        bits = index_to_bits(idx, n_index)
        # Map |bits> to |11..1> using X on 0-bits
        for qb, bit in enumerate(bits):
            if bit == 0:
                qc.x(qb)
        # Multi-controlled X on ancilla – with ancilla prepared in |-> this is a phase flip
        qc.mcx(list(range(n_index)), n_index)
        # Uncompute the X on 0-bits
        for qb, bit in enumerate(bits):
            if bit == 0:
                qc.x(qb)
    return qc.to_gate()


In [ ]:
def diffuser_gate(n_index: int):
    qc = QuantumCircuit(n_index, name="Diffuser")
    qc.h(range(n_index))
    qc.x(range(n_index))
    # Phase flip on |11..1>
    if n_index == 1:
        qc.z(0)
    else:
        qc.h(n_index - 1)
        qc.mcx(list(range(n_index - 1)), n_index - 1)
        qc.h(n_index - 1)
    qc.x(range(n_index))
    qc.h(range(n_index))
    return qc.to_gate()


In [ ]:
def run_grover_threshold(marked_indices, n_items: int, iterations: int = 1, shots: int = 1024):
    """Run Grover given a set of marked indices. Returns counts over index measurements."""
    n_index = num_qubits_for_n(n_items)
    qc = QuantumCircuit(n_index + 1, n_index, name="GroverThreshold")
    # Prepare index superposition
    qc.h(range(n_index))
    # Prepare ancilla in |-> state
    qc.x(n_index)
    qc.h(n_index)
    # Operators
    oracle = phase_oracle_gate(n_index, marked_indices)
    diffuser = diffuser_gate(n_index)
    
    for _ in range(iterations):
        qc.append(oracle, list(range(n_index + 1)))
        qc.append(diffuser, list(range(n_index)))

    # Measure index qubits
    qc.measure(list(range(n_index)), list(range(n_index)))
    backend = Aer.get_backend("qasm_simulator")
    tqc = transpile(qc, backend)
    result = backend.run(tqc, shots=shots).result()
    counts = result.get_counts()
    return counts, qc


## Example data (synthetic)
- Lower values are healthier.
- We will demonstrate threshold search (e.g., τ = 0.8) and minimum finding.

In [ ]:
pump_ids = [f"P{i:03d}" for i in range(8)]
time_of_use  = [0.10, 0.52, 0.33, 0.01, 0.72, 0.64, 0.28, 0.41]
max_current  = [0.21, 0.18, 0.11, 0.09, 0.85, 0.67, 0.44, 0.32]
vibration    = [0.03, 0.14, 0.59, 0.20, 0.22, 0.47, 0.18, 0.35]

N = len(pump_ids)
tau = 0.8  # threshold for maintenance candidates


## Threshold Grover (maintenance candidates)
We mark indices with values ≥ τ for a chosen variable (e.g., `max_current`).
- For demonstration, we compute the number of marked items M to pick a near-optimal Grover iteration count k ≈ π/4·√(N/M).
- In practice, when M is unknown, use BBHT (randomized iterations per round).


In [ ]:
values = max_current  # choose variable to analyze for thresholding
marked = [i for i, v in enumerate(values) if v >= tau]
M = len(marked)
print("Marked indices (v >= tau):", marked)

if M == 0:
    print("No pumps over threshold.")
else:
    # Choose iteration count (cap to avoid over-rotation in tiny examples)
    k = max(1, int(round((pi/4) * sqrt(N / M))))
    counts, qc = run_grover_threshold(marked, N, iterations=k, shots=2048)
    print("Grover iterations:", k)
    print("Top results:")
    for bitstring, c in Counter(counts).most_common(8):
        idx = int(bitstring, 2)
        print(bitstring, "-> index", idx, pump_ids[idx], "count", c)


## Simplified Dürr–Høyer minimum finding (healthiest pump)
This prototype uses an embedded-value oracle (mark all indices with value < current_threshold) and runs a few Grover iterations per round.
It is not a full implementation of the original paper but demonstrates the iterative idea for small N.

In [ ]:
def dh_min_index(values, shots=2048, max_rounds=5):
    N = len(values)
    n_index = num_qubits_for_n(N)
    # Start from a random index
    current = random.randrange(N)
    current_val = values[current]
    history = [(current, current_val)]

    for _ in range(max_rounds):
        marked = [j for j, v in enumerate(values) if v < current_val]
        if not marked:
            break
        M = len(marked)
        k = max(1, int(round((pi/4) * sqrt(N / M))))
        counts, _ = run_grover_threshold(marked, N, iterations=k, shots=shots)
        # Choose best candidate from measurement distribution
        candidates = [(int(bs, 2), c) for bs, c in counts.items()]
        candidates.sort(key=lambda x: x[1], reverse=True)
        cand_idx = candidates[0][0]
        cand_val = values[cand_idx]
        history.append((cand_idx, cand_val))
        if cand_val < current_val:
            current, current_val = cand_idx, cand_val
        else:
            # No improvement; stop
            break
    return current, history

# Test on a chosen variable (e.g., time_of_use)
values = time_of_use
idx_q, hist = dh_min_index(values, shots=2048, max_rounds=6)
idx_classical = min(range(N), key=lambda i: values[i])
print("Quantum (prototype) argmin index:", idx_q, pump_ids[idx_q], "value=", values[idx_q])
print("Classical argmin index:", idx_classical, pump_ids[idx_classical], "value=", values[idx_classical])
print("History (index, value):", hist)
